https://github.com/mapbox/rasterio

https://github.com/mapbox/rasterio-cookbook/blob/master/recipies/reproject.py

In [21]:
import os

import numpy as np
import pandas as pd

import gdal
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio import crs

import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import Point

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

In [22]:
location = 'mtbarker/'
shapefilename = 'mtbarker_shp/SA_VGO_Mt_Barker.shp'
polys = gpd.read_file(location + shapefilename)
poly_crs = polys.crs
polys

,OBJECTID,ASSNO,TENSEQNO,ASSNO_TENS,DATE_FROM,SVALNNO,RECSTAT,VALUATION_,VR_TYPE,LAND_USE_C,...,STREET_SUF,SUBURB,STATE_TERR,POST_CODE,ADDRESS_DE,LGA_NAME,Owner_Type,SHAPE_Leng,SHAPE_Area,geometry
0,186522,5818717000,0,58187170000000,2014-04-30,537218,0,5818717000,SCV,1100.0,...,None,LITTLEHAMPTON,SA,5250,LT 8 F157343,MOUNT BARKER,Private,123.464166,951.932433,"POLYGON ((1352071.336499996 1657656.3489, 1352..."
1,186523,5818755007,0,58187550070000,2014-05-01,68998,0,5818755007,SCV,1912.0,...,None,BLAKISTON,SA,5250,LT 10 F157345,MOUNT BARKER,Private,531.171415,15325.734412,"POLYGON ((1353543.670400001 1656976.2925, 1353..."
2,187385,5819148006,0,58191480060000,2014-06-06,537206,0,5819148006,SCV,1100.0,...,None,MOUNT BARKER,SA,5251,LT 96 F160273,MOUNT BARKER,Private,275.184428,4161.117028,"POLYGON ((1352509.290799998 1654864.4531, 1352..."
3,187386,5819149009,0,58191490090000,2014-06-06,68986,0,5819149009,SCV,1912.0,...,None,MOUNT BARKER,SA,5251,LT 95 F160272,MOUNT BARKER,Private,382.977676,8148.175459,"POLYGON ((1352442.957800001 1654754.206700001,..."
4,188068,5818773002,0,58187730020000,2014-05-01,32371,0,5818773002,SCV,1992.0,...,None,NAIRNE,SA,5252,LT 100 F157335,MOUNT BARKER,Business,2359.821272,334300.689494,"POLYGON ((1354817.406900004 1656079.921399999,..."
5,188069,581886000*,0,581886000*0000,2014-04-30,537237,0,581886000*,SCV,1100.0,...,None,LITTLEHAMPTON,SA,5250,LT 96 F157331,MOUNT BARKER,Private,298.995393,4046.793083,"POLYGON ((1353690.470899999 1656317.959000001,..."
6,192835,5811281005,0,58112810050000,2014-05-28,534243,0,5811281005,SCV,1100.0,...,None,MOUNT BARKER,SA,5251,LT 138 F9188,MOUNT BARKER,Private,135.456861,1095.230803,"POLYGON ((1351674.183399998 1653795.092399999,..."
7,192836,5811284003,0,58112840030000,2014-05-28,534245,0,5811284003,SCV,5390.0,...,None,MOUNT BARKER,SA,5251,LT 135 F9188,MOUNT BARKER,Associations,136.687945,1117.480736,"POLYGON ((1351606.891099997 1653807.2565, 1351..."
8,192837,5811287001,0,58112870010000,2014-05-28,534248,0,5811287001,SCV,1100.0,...,None,MOUNT BARKER,SA,5251,LT 157 F9188,MOUNT BARKER,Private,134.643492,1081.620770,"POLYGON ((1351693.864799999 1653759.3882, 1351..."
9,192838,5811291000,0,58112910000000,2014-05-28,534254,0,5811291000,SCV,1100.0,...,None,MOUNT BARKER,SA,5251,LT 155 F9188,MOUNT BARKER,Private,134.978169,1089.360862,"POLYGON ((1351765.502499998 1653795.528200001,..."


In [23]:
# https://github.com/mapbox/rasterio-cookbook/blob/master/recipies/reproject.py

rasterfilename = 'change_time.img'
reprojfile = 'change_reproj.img'
out = location + reprojfile 

dst_crs = poly_crs 

with rasterio.Env(CHECK_WITH_INVERT_PROJ=True):
    with rasterio.open(location + rasterfilename) as src:
        profile = src.profile

        # Calculate the ideal dimensions and transformation in the new crs
        dst_affine, dst_width, dst_height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)

        # update the relevant parts of the profile
        profile.update({
            'crs': dst_crs,
            'transform': dst_affine,
            'width': dst_width,
            'height': dst_height
        })
        # Reproject and write each band
        with rasterio.open(out, 'w', **profile) as dst:
            for i in range(1, src.count + 1):
                src_array = src.read(i)
                dst_array = np.empty((dst_height, dst_width), dtype='float32')

                reproject(
                    # Source parameters
                    source=src_array,
                    src_crs=src.crs,
                    src_transform=src.transform,
                    # Destination paramaters
                    destination=dst_array,
                    dst_transform=dst_affine,
                    dst_crs=dst_crs,
                    # Configuration
                    resampling=Resampling.nearest,
                    num_threads=2)
 
                dst.write(dst_array, i)


change = rasterio.open(location + reprojfile)
change

<open DatasetReader name='mtbarker/change_reproj.img' mode='r'>

In [24]:
dates = change.read()[0]
data = {'geometry': [], 'value':[]}
for j in range(len(dates)):
    for i in range(len(dates[0])):
        data['geometry'].append(Point(change.transform * (i + 0.5, j + .05)))
        data['value'].append(dates[j, i])
pixels = gpd.GeoDataFrame(data)
pixels.crs = change.crs
pixels['value'][pixels['value'] == 0].loc[:] = np.nan
pixels

,geometry,value
0,POINT (1350640.370867679 1656209.137793693),0.0
1,POINT (1350665.364147018 1656209.137793693),0.0
2,POINT (1350690.357426356 1656209.137793693),0.0
3,POINT (1350715.350705695 1656209.137793693),0.0
4,POINT (1350740.343985033 1656209.137793693),0.0
5,POINT (1350765.337264372 1656209.137793693),0.0
6,POINT (1350790.33054371 1656209.137793693),0.0
7,POINT (1350815.323823049 1656209.137793693),0.0
8,POINT (1350840.317102387 1656209.137793693),0.0
9,POINT (1350865.310381726 1656209.137793693),0.0


In [25]:
def checkPoints(leftcell, rightcell, bottomcell, topcell, i):
    l = len(dates)
    idx_to_check = []
    for j in range(topcell, bottomcell + 1):
        idx_to_check += list(range((j * l + leftcell), (j * l + rightcell + 1)))
    
    values = []
    for point in idx_to_check:
        if polys.loc[i,'geometry'].intersects(pixels.loc[point,'geometry']) and np.isfinite(pixels.loc[point,'value']):
            values.append(pixels.loc[point, 'value'])
    return values

bounds = polys.bounds

polys['count'] = 0
polys['sum'] = np.nan
polys['std_dev'] = np.nan
polys['max'] = np.nan
polys['min'] = np.nan


for i in bounds.index:
    leftx, bottomy, rightx, topy = bounds.loc[i].values
#     print(leftx, bottomy, rightx, topy)
    topcell, leftcell = change.index(leftx, topy)
    bottomcell, rightcell = change.index(rightx, bottomy)
#     print(topcell, bottomcell, rightcell, leftcell)
#     print(
    results[i] = []  
    if topcell < 0:
        topcell = 0
    if bottomcell < 0:
        bottomcell = 0
    if leftcell < 0:
        leftcell = 0
    if rightcell < 0:
        rightcell = 0
        
    h = len(dates)
    w = len(dates[0])
    if topcell > h:
        topcell = h
    if bottomcell > h:
        bottomcell = h +1
    if leftcell > w:
        leftcell = w
    if rightcell < w:
        rightcell = w + 1      
        
    results = checkPoints(leftcell, rightcell, bottomcell, topcell, i)
    
    if len(results) > 0:
        arr = np.array(results)
        arr = arr[arr > 0]
        if len(arr) > 0:
            polys.loc[i, 'count'] = len(arr)
            polys.loc[i, 'sum'] = arr.sum()
            polys.loc[i, 'std_dev'] = np.std(arr)
            polys.loc[i, 'max'] = arr.max()
            polys.loc[i, 'min'] = arr.min()
    
    if (i % 10) == 0:
        print("Polygon Number:", i, 'complete')


Polygon Number: 0
Polygon Number: 10
Polygon Number: 20
Polygon Number: 30
Polygon Number: 40
Polygon Number: 50
Polygon Number: 60
Polygon Number: 70
Polygon Number: 80
Polygon Number: 90
Polygon Number: 100
Polygon Number: 110
Polygon Number: 120
Polygon Number: 130
Polygon Number: 140
Polygon Number: 150
Polygon Number: 160
Polygon Number: 170
Polygon Number: 180
Polygon Number: 190
Polygon Number: 200
Polygon Number: 210
Polygon Number: 220
Polygon Number: 230
Polygon Number: 240
Polygon Number: 250
Polygon Number: 260
Polygon Number: 270
Polygon Number: 280
Polygon Number: 290
Polygon Number: 300
Polygon Number: 310
Polygon Number: 320
Polygon Number: 330
Polygon Number: 340
Polygon Number: 350
Polygon Number: 360
Polygon Number: 370
Polygon Number: 380
Polygon Number: 390
Polygon Number: 400
Polygon Number: 410
Polygon Number: 420
Polygon Number: 430
Polygon Number: 440
Polygon Number: 450
Polygon Number: 460
Polygon Number: 470
Polygon Number: 480
Polygon Number: 490
Polygon Num

Polygon Number: 3960
Polygon Number: 3970
Polygon Number: 3980
Polygon Number: 3990
Polygon Number: 4000
Polygon Number: 4010
Polygon Number: 4020
Polygon Number: 4030
Polygon Number: 4040
Polygon Number: 4050
Polygon Number: 4060
Polygon Number: 4070
Polygon Number: 4080
Polygon Number: 4090
Polygon Number: 4100
Polygon Number: 4110
Polygon Number: 4120
Polygon Number: 4130
Polygon Number: 4140
Polygon Number: 4150
Polygon Number: 4160
Polygon Number: 4170
Polygon Number: 4180
Polygon Number: 4190
Polygon Number: 4200
Polygon Number: 4210
Polygon Number: 4220
Polygon Number: 4230
Polygon Number: 4240
Polygon Number: 4250
Polygon Number: 4260
Polygon Number: 4270
Polygon Number: 4280
Polygon Number: 4290
Polygon Number: 4300
Polygon Number: 4310
Polygon Number: 4320
Polygon Number: 4330
Polygon Number: 4340
Polygon Number: 4350
Polygon Number: 4360
Polygon Number: 4370
Polygon Number: 4380
Polygon Number: 4390
Polygon Number: 4400
Polygon Number: 4410
Polygon Number: 4420
Polygon Numbe

Polygon Number: 7870
Polygon Number: 7880
Polygon Number: 7890
Polygon Number: 7900
Polygon Number: 7910
Polygon Number: 7920
Polygon Number: 7930
Polygon Number: 7940
Polygon Number: 7950
Polygon Number: 7960
Polygon Number: 7970
Polygon Number: 7980
Polygon Number: 7990
Polygon Number: 8000
Polygon Number: 8010
Polygon Number: 8020
Polygon Number: 8030
Polygon Number: 8040
Polygon Number: 8050
Polygon Number: 8060
Polygon Number: 8070
Polygon Number: 8080
Polygon Number: 8090
Polygon Number: 8100
Polygon Number: 8110
Polygon Number: 8120
Polygon Number: 8130
Polygon Number: 8140
Polygon Number: 8150
Polygon Number: 8160
Polygon Number: 8170
Polygon Number: 8180
Polygon Number: 8190
Polygon Number: 8200
Polygon Number: 8210
Polygon Number: 8220
Polygon Number: 8230
Polygon Number: 8240
Polygon Number: 8250
Polygon Number: 8260
Polygon Number: 8270
Polygon Number: 8280
Polygon Number: 8290
Polygon Number: 8300
Polygon Number: 8310
Polygon Number: 8320
Polygon Number: 8330
Polygon Numbe

{0: [[]],
 1: [[]],
 2: [[]],
 3: [[]],
 4: [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]],
 5: [[]],
 6: [[]],
 7: [[]],
 8: [[]],
 9: [[]],
 10: [[]],
 11: [[]],
 12: [[]],
 13: [[]],
 14: [[]],
 15: [[]],
 16: [[]],
 17: [[]],
 18: [[]],
 19: [[]],
 20: [[]],
 21: [[]],
 22: [[]],
 23: [[]],
 24: [[]],
 25: [[]],
 26: [[]],
 27: [[]],
 28: [[]],
 29: [[]],
 30: [[]],
 31: [[]],
 32: [[]],
 33: [[]],
 34: [[]],
 35: [[]],
 36: [[]],
 37: [[]],
 38: [[]],
 39: [[]],
 40: [[]],
 41: [[]],
 42: [[]],
 43: [[]],
 44: [[]],
 45: [[]],
 46: [[]],
 47: [[]],
 48: [[]],
 49: [[]],
 50: [[]],
 51: [[]],
 52: [[]],
 53: [[]],
 54: [[]],
 55: [[]],
 56: [[]],
 57: [[]],
 58: [[]],
 59: [[]],
 60: [[]],
 61: [[]],
 62: [[]],
 63: [[]],
 64: [[]],
 65: [[]],
 66: [[]],
 67: [[]],
 68: [[]],
 69: [[]],
 70: [[]],
 71: [[]],
 72: [[]],
 73: [[]],
 74: [[]],
 75: [[]],
 76: [[]],
 77: [[]],
 78: [[]],
 79: [[]],
 80: [[]],
 81: [[]],
 82: [[]],
 83: [[]],
 84: [[]],
 85: [[]],
 86: [[]],
 87: [[]],

In [31]:


for i in results.keys():
    if len(results) > 0:
        arr = np.array(results[i])
        arr = arr[arr > 0]
        if len(arr) > 0:
            polys.loc[i, 'count'] = len(arr)
            polys.loc[i, 'sum'] = arr.sum()
            polys.loc[i, 'std_dev'] = np.std(arr)
            polys.loc[i, 'max'] = arr.max()
            polys.loc[i, 'min'] = arr.min()

In [32]:
polys

,OBJECTID,ASSNO,TENSEQNO,ASSNO_TENS,DATE_FROM,SVALNNO,RECSTAT,VALUATION_,VR_TYPE,LAND_USE_C,...,LGA_NAME,Owner_Type,SHAPE_Leng,SHAPE_Area,geometry,count,sum,std_dev,max,min
0,186522,5818717000,0,58187170000000,2014-04-30,537218,0,5818717000,SCV,1100.0,...,MOUNT BARKER,Private,123.464166,951.932433,"POLYGON ((1352071.336499996 1657656.3489, 1352...",0.0,0.0,NaN,NaN,NaN
1,186523,5818755007,0,58187550070000,2014-05-01,68998,0,5818755007,SCV,1912.0,...,MOUNT BARKER,Private,531.171415,15325.734412,"POLYGON ((1353543.670400001 1656976.2925, 1353...",NaN,NaN,NaN,NaN,NaN
2,187385,5819148006,0,58191480060000,2014-06-06,537206,0,5819148006,SCV,1100.0,...,MOUNT BARKER,Private,275.184428,4161.117028,"POLYGON ((1352509.290799998 1654864.4531, 1352...",NaN,NaN,NaN,NaN,NaN
3,187386,5819149009,0,58191490090000,2014-06-06,68986,0,5819149009,SCV,1912.0,...,MOUNT BARKER,Private,382.977676,8148.175459,"POLYGON ((1352442.957800001 1654754.206700001,...",NaN,NaN,NaN,NaN,NaN
4,188068,5818773002,0,58187730020000,2014-05-01,32371,0,5818773002,SCV,1992.0,...,MOUNT BARKER,Business,2359.821272,334300.689494,"POLYGON ((1354817.406900004 1656079.921399999,...",NaN,NaN,NaN,NaN,NaN
5,188069,581886000*,0,581886000*0000,2014-04-30,537237,0,581886000*,SCV,1100.0,...,MOUNT BARKER,Private,298.995393,4046.793083,"POLYGON ((1353690.470899999 1656317.959000001,...",NaN,NaN,NaN,NaN,NaN
6,192835,5811281005,0,58112810050000,2014-05-28,534243,0,5811281005,SCV,1100.0,...,MOUNT BARKER,Private,135.456861,1095.230803,"POLYGON ((1351674.183399998 1653795.092399999,...",NaN,NaN,NaN,NaN,NaN
7,192836,5811284003,0,58112840030000,2014-05-28,534245,0,5811284003,SCV,5390.0,...,MOUNT BARKER,Associations,136.687945,1117.480736,"POLYGON ((1351606.891099997 1653807.2565, 1351...",NaN,NaN,NaN,NaN,NaN
8,192837,5811287001,0,58112870010000,2014-05-28,534248,0,5811287001,SCV,1100.0,...,MOUNT BARKER,Private,134.643492,1081.620770,"POLYGON ((1351693.864799999 1653759.3882, 1351...",NaN,NaN,NaN,NaN,NaN
9,192838,5811291000,0,58112910000000,2014-05-28,534254,0,5811291000,SCV,1100.0,...,MOUNT BARKER,Private,134.978169,1089.360862,"POLYGON ((1351765.502499998 1653795.528200001,...",NaN,NaN,NaN,NaN,NaN


In [ ]:
# polys['count'] = 0
# polys['sum'] = 0

# use results to get states and make new fields for each poly to store the stats

In [158]:
for poly in polys.index:
    for point in pixels.index:
        if polys.loc[poly,'geometry'].intersects(pixels.loc[point,'geometry']) and np.isfinite(pixels.loc[point,'geometry']):
            polys.loc[poly, ['count']] += 1
            polys.loc[poly, ['count']] += pixels.loc[point, 'value']
    if poly % 10 == 0:
        print("Polygon Number:", poly)

Polygon Number: 0
Polygon Number: 10
Polygon Number: 20
Polygon Number: 30
Polygon Number: 40
Polygon Number: 50
Polygon Number: 60
Polygon Number: 70
Polygon Number: 80
Polygon Number: 90
Polygon Number: 100
Polygon Number: 110


KeyboardInterrupt: 

In [268]:
def makePoly(row):
    poly = Polygon([(row['minx'], row['miny']), (row['maxx'], row['miny']),
                    (row['maxx'], row['maxy']), (row['minx'], row['maxy'])])
    return poly

bounds = pd.DataFrame({'minx':[1, 6, 4],'miny':[1,3,6], 'maxx':[4, 9, 6], 'maxy':[5, 5, 8]})
bounds



,maxx,maxy,minx,miny
0,4,5,1,1
1,9,5,6,3
2,6,8,4,6


In [278]:
polys = bounds.copy()
polys['type'] = ['house', 'unit', 'land']
poly = gpd.GeoDataFrame(polys)
polys['geometry'] = polys.apply(lambda row: makePoly(row), axis = 1)

polys


,maxx,maxy,minx,miny,type,geometry
0,4,5,1,1,house,"POLYGON ((1 1, 4 1, 4 5, 1 5, 1 1))"
1,9,5,6,3,unit,"POLYGON ((6 3, 9 3, 9 5, 6 5, 6 3))"
2,6,8,4,6,land,"POLYGON ((4 6, 6 6, 6 8, 4 8, 4 6))"


In [254]:
dates = np.arange(100).reshape(10,10) + .5
dates

array([[  0.5,   1.5,   2.5,   3.5,   4.5,   5.5,   6.5,   7.5,   8.5,
          9.5],
       [ 10.5,  11.5,  12.5,  13.5,  14.5,  15.5,  16.5,  17.5,  18.5,
         19.5],
       [ 20.5,  21.5,  22.5,  23.5,  24.5,  25.5,  26.5,  27.5,  28.5,
         29.5],
       [ 30.5,  31.5,  32.5,  33.5,  34.5,  35.5,  36.5,  37.5,  38.5,
         39.5],
       [ 40.5,  41.5,  42.5,  43.5,  44.5,  45.5,  46.5,  47.5,  48.5,
         49.5],
       [ 50.5,  51.5,  52.5,  53.5,  54.5,  55.5,  56.5,  57.5,  58.5,
         59.5],
       [ 60.5,  61.5,  62.5,  63.5,  64.5,  65.5,  66.5,  67.5,  68.5,
         69.5],
       [ 70.5,  71.5,  72.5,  73.5,  74.5,  75.5,  76.5,  77.5,  78.5,
         79.5],
       [ 80.5,  81.5,  82.5,  83.5,  84.5,  85.5,  86.5,  87.5,  88.5,
         89.5],
       [ 90.5,  91.5,  92.5,  93.5,  94.5,  95.5,  96.5,  97.5,  98.5,
         99.5]])